<h2>FT004 - Cuentas por Pagar </h2>
<p> En este apartado se trata la base de datos FT004 </p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [2]:
# Se carga el FT005 completo desde el archivo .dta
FT004_2021_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT004 20202021.dta')
# Se carga el FT004 para diciembre de 2019 desde el archivo .dta
FT004_2019 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT004 122019.dta')
# Se concatenan la bases de datos 
frames = [FT004_2021_2021, FT004_2019]
FT004 = pd.concat(frames, ignore_index=True)
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT004_2021_2021, FT004_2019, frames]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\stata.py:1433: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\stata.py:1433: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [3]:
# Se carga el FT004 completo desde el archivo .dta
FT004 = pd.read_csv('C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Bases_FT/BD_actualizadas/FT004_1219_0521.csv', dtype={'Nit':object, 'Año': object, 'Periodo': object})

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004.rename(columns={'Nit': 'nit_eps', 'Año': 'ano', 'Periodo': 'mes', 'lineaNegocio': 'linea','tipoIdAcreedor': 'tipoidacreedor', 'idAcreedor': 'nitproveedor', 'dvAcreedor': 'DVacreedor', 'conceptoAcreencia': 'concepto', 'cxpNoVencidas': 'cxpnovencidas', 'cxpMora30dias': 'cxp30','cxpMora60dias': 'cxp60', 'cxpMora90dias': 'cxp90', 'cxpMora180dias': 'cxp180', 'cxpMora360dias': 'cxp360','cxpMoraMayor360dias': 'cxpmayor360'}, inplace=True)

In [5]:
# Se transforman los datos para su correcta identificación
FT004[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoidacreedor',
       'nitproveedor', 'DVacreedor', 'nombreAcreedor', 'actividadAcreedor',
       'concepto', 'medicionPosterior']] = FT004[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoidacreedor',
       'nitproveedor', 'DVacreedor', 'nombreAcreedor', 'actividadAcreedor',
       'concepto', 'medicionPosterior']].astype(str)

FT004[['cxpnovencidas', 'cxp30', 'cxp60',
       'cxp90', 'cxp180', 'cxp360', 'cxpmayor360', 'ajuste', 'saldo']] = FT004[['cxpnovencidas', 'cxp30', 'cxp60',
       'cxp90', 'cxp180', 'cxp360', 'cxpmayor360', 'ajuste', 'saldo']].astype(float)

In [6]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT004 = FT004.loc[FT004['linea'] == '1']

In [7]:
# Se crea la columna Cuentas por Pagar (CXP) sumando las columnas cxpnovencidas, cxp30, cxp60, cxp90, cxp180, cxp360 y cxpmayor360
CXP = FT004['cxpnovencidas'] + FT004['cxp30'] + FT004['cxp60'] + FT004['cxp90'] + FT004['cxp180'] + FT004['cxp360'] + FT004['cxpmayor360']
FT004['CXP'] = CXP

In [8]:
# Para hacer la suma de CXC por NIT EPS, tipo de proveedor, NIT proveedor, Mes y Año
FT004 = FT004.groupby(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano']).agg({'CXP': 'sum'}).reset_index()

In [9]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de deudor, NIT proveedor, Mes y Año. Para esto se cuenta el número de duplicados
duplicadosFT004 = FT004[FT004.duplicated(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT004.nit_eps.count(), sep='\n')

El número de registros duplicados es:
0


In [10]:
# Se carga la base de datos cuenta2.dta para su posterior pega
cuentas2 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cuentas2.dta')

In [11]:
# Se tranforma el formato de las columnas del dataframe cuentas2
cuentas2[['v2', 'nitproveedor']] = cuentas2[['v2', 'nitproveedor']].astype(str)

In [12]:
# Se comprueba que no haya duplicados cuando se filtra por Nitproveedor. Para esto se cuenta el número de duplicados
duplicados_cuentas2 = cuentas2[cuentas2.duplicated(['nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicados_cuentas2.nitproveedor.count(), sep='\n')

El número de registros duplicados es:
0


In [13]:
# Se unen las dos bases de datos para obtener la información de FT004 con cuenta2
FT004= pd.merge(FT004, cuentas2, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

<p> Esto corresponde a la cuenta del pasivo cuando se agregan las cuentas por pagar. Por tal motivo, se elimina ya que no se pueden identificar los proveedores. </p>

In [14]:
# Se encuentran las observaciones que cumplen las condiciones anteriores
df_filtered = FT004[(FT004['tipoidacreedor'] == 'OT') & (FT004['v2'] == 'cuenta')].index

In [15]:
# Se eliminan las observaciones que cumplen las condiciones anteriores
FT004 = FT004.drop(df_filtered)

In [16]:
# Se genera una nueva columna extrayendo el primer caracter del ID nitproveedor
FT004['d1'] = FT004['nitproveedor'].str[:1]

In [17]:
# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT004['largo'] = FT004.nitproveedor.str.len()

In [18]:
# Se limpia la clasificación por Tipo de acreedor
FT004['tipoidacreedor_corregido'] = FT004['tipoidacreedor'] # Se crea la columna con categoria NA como base
# Condiciones
FT004.loc[
       (FT004['tipoidacreedor_corregido'] == 'NA') &
       (FT004['largo'] == 10) & # Para signar el tipo de acreedor CC
       (FT004['d1'] == '1'), 
       'tipoidacreedor_corregido'] = 'CC' 
FT004.loc[(FT004['tipoidacreedor_corregido'] == 'NA') &
       (FT004['largo'] == 9) & # Para signar el tipo de acreedor NI
       ((FT004['d1'] == '7') |
       (FT004['d1'] == '8') |
       (FT004['d1'] == '9')),  
       'tipoidacreedor_corregido'] = 'NI' 
FT004.loc[(FT004['tipoidacreedor_corregido'] == 'NA') &
       (FT004['tipoidacreedor_corregido'] != 'NI'), # Para completar el tipo de acreedor CC
       'tipoidacreedor_corregido'] = 'CC' 

In [19]:
# Para conocer las diferencias entre las correciones
print(FT004.tipoidacreedor.value_counts())
print(FT004.tipoidacreedor_corregido.value_counts())

CC    2709239
NI    2591695
OT       4356
CE       3195
Name: tipoidacreedor, dtype: int64
CC    2709239
NI    2591695
OT       4356
CE       3195
Name: tipoidacreedor_corregido, dtype: int64


In [20]:
# Para hacer la suma de CXP por NIT EPS, tipo de acreedor corregido, NIT proveedor, Mes y Año
FT004 = FT004.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).agg({'CXP': 'sum'}).reset_index()

In [21]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
#FT004.rename(columns={'tipoidacreedor_corregido': 'tipoidacreedor'}, inplace=True)

In [22]:
# Se transforman las columnas de mes y año para su correcta identificación
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(int)

In [23]:
del [CXP, cuentas2, df_filtered, duplicados_cuentas2, duplicadosFT004]

In [24]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT004 = FT004.set_index('mes').groupby(['nit_eps', 'nitproveedor', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'nitproveedor', 'ano'], 1).reset_index()

In [25]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT004[
    (FT004['ano'] == 2019) & (FT004['mes'] == 1) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 2) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 3) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 4) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 5) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 6) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 7) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 8) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 9) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 10) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 11)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004 = FT004.drop(df_filtered)

In [26]:
del [fecha_ideal]

In [27]:
# Se completan los años y los meses para las observaciones que no presentaron ninguna observación en un año de interés. Por ejemplo, a aquellos sujetos que reportaron cifras en 2019, pero no en 2020, se les completa el año 2020 para poder calcular la diferencia entre diciembre de 2019 y enero de 2020
# Se convierte la variable en mes y año
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(float)
FT004['periodo'] = pd.to_datetime(FT004.ano*10000 + FT004.mes*100 + 1, format='%Y%m%d')
fecha_ideal = pd.date_range('12-01-2019','05-01-2021',freq='M')

In [28]:
FT004_dif = FT004.groupby(['nit_eps','nitproveedor', pd.Grouper(key='periodo', freq='M')])[['CXP']].sum().reset_index()

In [29]:
FT004_dif = FT004_dif.set_index('periodo').groupby(['nit_eps', 'nitproveedor']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'nitproveedor'], 1).reset_index()

In [30]:
FT004_dif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11484214 entries, 0 to 11484213
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   nit_eps       object        
 1   nitproveedor  object        
 2   level_2       datetime64[ns]
 3   CXP           float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 350.5+ MB


In [31]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_dif.rename(columns={'level_2': 'periodo'}, inplace=True)

In [32]:
FT004_dif = FT004_dif.set_index(['nit_eps', 'nitproveedor', 'periodo'])

In [33]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT004_dif = FT004_dif.diff().fillna(0).reset_index()

In [34]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_dif.rename(columns={'CXP': 'DCXPmes'}, inplace=True)

#  Se extrae el mes y el año para porterior left join con el agregado
FT004_dif['ano'] = FT004_dif['periodo'].dt.year
FT004_dif['mes'] = FT004_dif['periodo'].dt.month

#Se transforman las variables a STR para mejorar su manipulación
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(int)

# Se eliminan las columnas que no son de interés
FT004 = FT004.drop(['periodo'], axis=1)
FT004_dif = FT004_dif.drop(['periodo'], axis=1)

In [35]:
# Se unen las dos bases de datos para obtener la información de webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y FT005 en una sola base de datos. Desagregando el valor de VCausado y el VPagado para cada mes
FT004 = pd.merge(FT004_dif, FT004, how='left', left_on=['nit_eps', 'nitproveedor', 'mes', 'ano'], right_on=['nit_eps', 'nitproveedor', 'mes', 'ano'])

In [36]:
del [df_filtered, fecha_ideal]

In [37]:
# Se convierte la variable en mes y año
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(float).astype(int)

In [38]:
# Se agrega la información por NIT_EPS, Nitproveedor, noID, Departamento, Ano, Mes para las variables VPagado, VCausado, PW, Giro, No_UPC, Compra_Cartera
FT004 = FT004.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).agg({'CXP':'sum', 'DCXPmes':'sum'}).reset_index()

In [39]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV

# FT004_V2.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/carteraprestador_2020_2021.xlsx', index = False)
FT004.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/carteraprestador_2020_ene_mayo_2021.csv', index = False, encoding='utf-8-sig')

In [40]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT004, FT004_dif]